In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import re

In [2]:
# get a list of unique zip codes from trulia data set

df = pd.read_csv('../data/trulia_cleaned.csv')
zip_codes = list(set(df['zip_code']))
print("Number of unique zip_codes:", len(zip_codes))

Number of unique zip_codes: 168


In [6]:
# make sure all zip_codes are string objects
zip_codes = [str(zip_code) for zip_code in df.zip_code]
zip_codes = list(set(zip_codes))

In [12]:
test = zip_codes[:10]
test

['10030',
 '11226',
 '10305',
 '10280',
 '11208',
 '11362',
 '10469',
 '10282',
 '11418',
 '10471']

In [13]:
# create a function that returns all the tables from the income page

def income_page(zipcode):
    url = f"https://www.incomebyzipcode.com/newyork/{zipcode}"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    income_page = response.text
    soup = BeautifulSoup(income_page,"html5lib")
    tables = soup.find_all("table")
    return(tables)

In [22]:
test = income_page(11249)

In [23]:
test

[]

In [26]:
# no data is available for 11249 zip code - remove
i = zip_codes.index("11249")
zip_codes.pop(i)

'11249'

In [29]:
# get median income data

income_data = pd.DataFrame(index=zip_codes,columns=['median_income','pct_high_income'])

for zipcode in zip_codes:
    tables = income_page(int(zipcode))
    
    # get median income
    rows_mi=[row for row in tables[0].find_all('tr')]
    med_inc = [el for el in rows_mi[1].find_all('td')]
    med_inc = med_inc[0].text.replace(',','')
    med_inc = int(med_inc.replace('$',''))
    
    # get pct high income households - what is HI 
    rows_hi=[row for row in tables[3].find_all('tr')]
    percents = [el for el in rows_hi[1].find_all('td')]
    pct = float(percents[0].text.replace('%',''))
    
    # fill out the dataframe
    income_data.at[zipcode, 'median_income'] = med_inc
    income_data.at[zipcode, 'pct_high_income'] = pct

In [30]:
income_data

,median_income,pct_high_income
10030,33720,4
11226,45031,2.6
10305,74063,6
10280,146875,37.6
11208,37640,1.8
11362,82827,8.9
10469,57124,3.3
10282,250001,59.9
11418,63079,4.3
10471,81594,11.7


In [32]:
# write the dataframe into a file
income_data.to_csv('../data/income_data_by_zip.csv')